In [1]:
import numpy as np
from pathlib import Path
from astropy import stats
from astropy import units as u
from astropy.modeling import models, fitting

from keckdata import KeckData, KeckDataList, fits_reader, analysis

%matplotlib inline
from matplotlib import pyplot as plt

/Users/jwalawender/anaconda/lib/python3.6/site-packages/scipy/__init__.py:115: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.11.3)
  UserWarning)


In [2]:
p_2019oct11 = Path('/Volumes/JOSHW/HIRES/2019oct11')
files = sorted([f for f in p_2019oct11.glob('*fits')])
p_2019oct12 = Path('/Volumes/JOSHW/HIRES/2019oct12')
files.extend( sorted([f for f in p_2019oct12.glob('*fits')]) )
kdl_biasesdarks = KeckDataList(files, verbose=False)

p_2019oct19 = Path('/Volumes/JOSHW/HIRES/2019oct19')
# files = sorted([f for f in p_2019oct19.glob('hires00[02][1-6].fits')])
files = sorted([f for f in p_2019oct19.glob('hires*.fits')])
kdl_flats = KeckDataList(files, verbose=False)

In [3]:
kdl_flats.len

36

In [4]:
mb = analysis.make_master_bias(kdl_biasesdarks)

2019-10-27 23:21:43     INFO: Making master bias from 13 frames
2019-10-27 23:21:43     INFO: Making master bias for each of 3 extensions
2019-10-27 23:22:15     INFO:   Done


In [5]:
RN = analysis.determine_read_noise(kdl_biasesdarks, master_bias=mb, plot=False)

2019-10-27 23:22:15     INFO: Determining read noise
2019-10-27 23:22:15     INFO:   Checking that all inputs are BIAS frames
2019-10-27 23:22:15     INFO:   Found 7 biases
2019-10-27 23:22:15     INFO: Making master bias from 6 frames
2019-10-27 23:22:15     INFO: Making master bias for each of 3 extensions
2019-10-27 23:22:37     INFO:   Done
2019-10-27 23:22:37     INFO:   Determining read noise for each of 3 extensions
2019-10-27 23:22:41     INFO:   Read Noise is 1.32 adu for extension 1
2019-10-27 23:22:46     INFO:   Read Noise is 1.27 adu for extension 2
2019-10-27 23:22:50     INFO:   Read Noise is 1.31 adu for extension 3


In [6]:
RN

<Quantity [ 1.32449115, 1.26762389, 1.30709388] adu>

In [7]:
dc = analysis.determine_dark_current(kdl_biasesdarks, master_bias=mb, plot=False)

In [8]:
dc

In [ ]:
g = analysis.determine_gain(kdl_flats, master_bias=mb, read_noise=RN, plot=True)

2019-10-27 23:22:50     INFO: Determining gain
2019-10-27 23:22:50     INFO: Step 1: Get image statistics for 36 flat frames
2019-10-27 23:22:50     INFO: Each frame has 3 extensions


In [ ]:
g